In [9]:
from xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure
from plotly.offline import init_notebook_mode, iplot
import networkx as nx
import pandas as pd
import re
import numpy as np
import os
import torch
import torch_geometric as pyg
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import periodictable as pt
import glob



In [61]:
def understand_mol_graph(path):
    # Parse the .xyz file
    with open(path, 'r') as file:
        lines = file.readlines()[2:]  # Skip the first two lines
        atoms = [tuple(map(float, line.split()[1:4])) for line in lines]  # Get the coordinates

    # Create the MolGraph object
    mg = MolGraph()
    mg.read_xyz(path)
    
    # Plot the molecular graph
    fig = to_plotly_figure(mg)
    iplot(fig)

    # Convert the molecular graph to the NetworkX graph
    G = to_networkx_graph(mg)
    print(G.nodes(data=True))
    print(G.edges(data=True))
    print(len(G.nodes))

    # Compare the atoms to the nodes in the graph
    graph_nodes = [data['xyz'] for node, data in G.nodes(data=True)]
    missing_atoms = set(atoms) - set(graph_nodes)
    print(f"Missing coordinates: {missing_atoms}")

understand_mol_graph('data/xyz_molsimp/monosubstituted_0001.xyz')

[(np.int64(0), {'element': 'Fe', 'xyz': (0.0, 0.0, 0.0)}), (np.int64(1), {'element': 'B', 'xyz': (0.0, 2.1, -0.0)}), (np.int64(20), {'element': 'O', 'xyz': (2.08, -0.0, -0.0)}), (np.int64(23), {'element': 'O', 'xyz': (-0.0, -2.08, 0.0)}), (np.int64(26), {'element': 'O', 'xyz': (-2.08, 0.0, 0.0)}), (np.int64(29), {'element': 'O', 'xyz': (0.0, 0.0, 2.08)}), (np.int64(32), {'element': 'O', 'xyz': (0.0, -0.0, -2.08)}), (np.int64(2), {'element': 'N', 'xyz': (-1.315822, 2.875, 0.265496)}), (np.int64(5), {'element': 'N', 'xyz': (1.315822, 2.875, -0.265496)}), (np.int64(3), {'element': 'C', 'xyz': (-1.629548, 3.720125, -0.895642)}), (np.int64(4), {'element': 'C', 'xyz': (-2.411063, 1.919746, 0.486484)}), (np.int64(8), {'element': 'H', 'xyz': (-1.746782, 3.104968, -1.763247)}), (np.int64(9), {'element': 'H', 'xyz': (-2.537891, 4.255123, -0.712365)}), (np.int64(10), {'element': 'H', 'xyz': (-0.832329, 4.415444, -1.056493)}), (np.int64(11), {'element': 'H', 'xyz': (-2.528296, 1.304585, -0.381118)

In [62]:
electronegativities = {
    "H": 2.20, "He": None,
    "Li": 0.98, "Be": 1.57, "B": 2.04, "C": 2.55, "N": 3.04, "O": 3.44, "F": 3.98, "Ne": None,
    "Na": 0.93, "Mg": 1.31, "Al": 1.61, "Si": 1.90, "P": 2.19, "S": 2.58, "Cl": 3.16, "Ar": None,
    "K": 0.82, "Ca": 1.00, "Sc": 1.36, "Ti": 1.54, "V": 1.63, "Cr": 1.66, "Mn": 1.55, "Fe": 1.83,
    "Co": 1.88, "Ni": 1.91, "Cu": 1.90, "Zn": 1.65, "Ga": 1.81, "Ge": 2.01, "As": 2.18, "Se": 2.55,
    "Br": 2.96, "Kr": 3.00,
    "Rb": 0.82, "Sr": 0.95, "Y": 1.22, "Zr": 1.33, "Nb": 1.6, "Mo": 2.16, "Tc": 1.9, "Ru": 2.2,
    "Rh": 2.28, "Pd": 2.20, "Ag": 1.93, "Cd": 1.69, "In": 1.78, "Sn": 1.96, "Sb": 2.05, "Te": 2.1,
    "I": 2.66, "Xe": 2.6,
    "Cs": 0.79, "Ba": 0.89, "La": 1.10, "Ce": 1.12, "Pr": 1.13, "Nd": 1.14, "Pm": None, "Sm": 1.17,
    "Eu": None, "Gd": 1.20, "Tb": None, "Dy": 1.22, "Ho": 1.23, "Er": 1.24, "Tm": 1.25, "Yb": None,
    "Lu": 1.27, "Hf": 1.3, "Ta": 1.5, "W": 2.36, "Re": 1.9, "Os": 2.2, "Ir": 2.20, "Pt": 2.28,
    "Au": 2.54, "Hg": 2.00, "Tl": 1.62, "Pb": 2.33, "Bi": 2.02, "Po": 2.0, "At": 2.2, "Rn": None,
    "Fr": 0.7, "Ra": 0.9, "Ac": 1.1, "Th": 1.3, "Pa": 1.5, "U": 1.38, "Np": 1.36, "Pu": 1.28,
    "Am": 1.3, "Cm": 1.3, "Bk": 1.3, "Cf": 1.3, "Es": 1.3, "Fm": 1.3, "Md": 1.3, "No": 1.3,
    "Lr": 1.3
}


def get_element_properties(element_symbol):
    element = pt.elements.symbol(element_symbol)
    
    # Normalize atomic number and weight
    normalized_atomic_number = element.number / 118.0
    normalized_weight = element.mass / 294.0

    electronegativity = electronegativities[element_symbol]
    atomic_radius = element.covalent_radius

    return normalized_atomic_number, normalized_weight, electronegativity, atomic_radius

In [63]:
for element in pt.elements:
    print(element.symbol, element.number, element.mass, element.covalent_radius)

n 0 1.00866491597 0.2
H 1 1.00794 0.31
He 2 4.002602 0.28
Li 3 6.941 1.28
Be 4 9.012182 0.96
B 5 10.811 0.84
C 6 12.0107 0.76
N 7 14.0067 0.71
O 8 15.9994 0.66
F 9 18.9984032 0.57
Ne 10 20.1797 0.58
Na 11 22.98977 1.66
Mg 12 24.305 1.41
Al 13 26.981538 1.21
Si 14 28.0855 1.11
P 15 30.973761 1.07
S 16 32.065 1.05
Cl 17 35.453 1.02
Ar 18 39.948 1.06
K 19 39.0983 2.03
Ca 20 40.078 1.76
Sc 21 44.95591 1.7
Ti 22 47.867 1.6
V 23 50.9415 1.53
Cr 24 51.9961 1.39
Mn 25 54.938049 1.39
Fe 26 55.845 1.32
Co 27 58.9332 1.26
Ni 28 58.6934 1.24
Cu 29 63.546 1.32
Zn 30 65.409 1.22
Ga 31 69.723 1.22
Ge 32 72.64 1.2
As 33 74.9216 1.19
Se 34 78.96 1.2
Br 35 79.904 1.2
Kr 36 83.798 1.16
Rb 37 85.4678 2.2
Sr 38 87.62 1.95
Y 39 88.90585 1.9
Zr 40 91.224 1.75
Nb 41 92.90638 1.64
Mo 42 95.94 1.54
Tc 43 98 1.47
Ru 44 101.07 1.46
Rh 45 102.9055 1.42
Pd 46 106.42 1.39
Ag 47 107.8682 1.45
Cd 48 112.411 1.44
In 49 114.818 1.42
Sn 50 118.71 1.39
Sb 51 121.76 1.39
Te 52 127.6 1.38
I 53 126.90447 1.39
Xe 54 131.293 1

In [64]:
get_element_properties('Co')

(0.2288135593220339, 0.2004530612244898, 1.88, 1.26)

In [12]:
def to_pyg_graph_triple(name, gen_xyz_path, gen_csv_path):
    # Read the csv file
    try:
        df = pd.read_csv(f"{gen_csv_path}.csv")
    except FileNotFoundError:
        print(f"No csv file found at {gen_csv_path}.csv")
        return None
    
    # Filter the csv data to get the row associated with the name
    row = df[df['name'] == name]
    
    # Check if the row is empty
    if row.empty:
        print(f"No data found for name {name} in the csv file {gen_csv_path}.csv")
        return None
    
    # Extract the first two letters of the name which symbolize the element
    element = re.match(r"^[A-Za-z]{2}", name).group()
    
    # Construct the rest of the name
    rest_of_the_name = re.sub(r"^[A-Za-z]{2}", "", name)
    
    # Construct the filenames for the xyz data
    filenames = [f"{element}_II{rest_of_the_name}_{state}.xyz" for state in ['HS', 'LS']]
    filenames.extend([f"{element}_III{rest_of_the_name}_{state}.xyz" for state in ['FS', 'LS']])
    
    data_list = []
    for filename in filenames:
        try:
            # Create the MolGraph object for the file
            mg = MolGraph()
            mg.read_xyz(f"{gen_xyz_path}/{filename}")

            # Convert the molecular graph to the NetworkX graph
            G = to_networkx_graph(mg)

            # Get element properties for each node and store them in the node features
            for node in G.nodes:
                element_symbol = G.nodes[node]['element']
                G.nodes[node]['element_properties'] = get_element_properties(element_symbol)

            # Convert the NetworkX graph to the PyG graph
            data = from_networkx(G, group_node_attrs=['xyz', 'element_properties'], group_edge_attrs=['length'])

            if data is not None:
                data_list.append(data)
        except FileNotFoundError:
            print(f"No xyz file found at {gen_xyz_path}/{filename}")
    
    # Concatenate the PyG graph data
    x = [d.x for d in data_list if d.x is not None]
    edge_index = [d.edge_index for d in data_list if d.edge_index is not None]
    edge_attr = [d.edge_attr for d in data_list if d.edge_attr is not None]

    if x and edge_index and edge_attr:  # Check if the lists are not empty
        data = Data(x=torch.cat(x, dim=0),
                    edge_index=torch.cat(edge_index, dim=1),
                    edge_attr=torch.cat(edge_attr, dim=0))
    else:
        print("No valid data found in the files.")
        return None
    
    # Get the last 7 columns of the filtered row
    output_data = row.iloc[:, -7:]
    
    # Add the output data to the PyG graph data
    for col in output_data.columns:
        data[col] = output_data[col].values[0]
    
    return data

In [13]:
def to_pyg_graph_single(name, gen_xyz_path, gen_csv_path):
    # Read the csv file
    try:
        df = pd.read_csv(f"{gen_csv_path}.csv")
    except FileNotFoundError:
        print(f"No csv file found at {gen_csv_path}.csv")
        return None
    
    # Filter the csv data to get the row associated with the name
    row = df[df['name'] == name]
    
    # Check if the row is empty
    if row.empty:
        print(f"No data found for name {name} in the csv file {gen_csv_path}.csv")
        return None
    
    # Extract the first two letters of the name which symbolize the element
    element = re.match(r"^[A-Za-z]{2}", name).group()
    
    # Construct the rest of the name
    rest_of_the_name = re.sub(r"^[A-Za-z]{2}", "", name)
    
    # Construct the filenames for the xyz data
    filenames = [f"{element}_II{rest_of_the_name}_LS.xyz", f"{element}_II{rest_of_the_name}_FS.xyz", f"{element}_III{rest_of_the_name}_LS.xyz", f"{element}_III{rest_of_the_name}_FS.xyz"]
    
    graph_list = []
    for filename in filenames:
        filepath = f"{gen_xyz_path}/{filename}"
        if os.path.exists(filepath):
            # Create the MolGraph object for the file
            mg = MolGraph()
            mg.read_xyz(filepath)

            # Convert the molecular graph to the NetworkX graph
            G = to_networkx_graph(mg)

            # Get element properties for each node and store them in the node features (only for the first graph)
            if not graph_list:
                for node in G.nodes:
                    element_symbol = G.nodes[node]['element']
                    element_properties = get_element_properties(element_symbol)
                    G.nodes[node]['element_properties'] = element_properties

            graph_list.append(G)
        else:
            print(f"No xyz file found at {filepath}")
    
    if not graph_list:
        print(f"No valid grapFS found for {name}")
        return None
    
    # Concatenate the xyz data from the available graphs
    merged_graph = graph_list[0]
    for i, G in enumerate(graph_list[1:], start=1):
        for node in G.nodes:
            merged_graph.nodes[node][f'xyz_{i}'] = G.nodes[node]['xyz']
    
    # Create the PyTorch Geometric Data object
    data = Data()
    
    # Define the desired order of node attributes
    node_attr_order = ['element_properties', 'xyz'] + [f'xyz_{i}' for i in range(1, len(graph_list))]
    
    # Set the node features
    node_attrs = []
    for attr in node_attr_order:
        if attr in merged_graph.nodes[0]:
            if isinstance(merged_graph.nodes[0][attr], (int, float)):
                node_attrs.append(torch.tensor([[merged_graph.nodes[node][attr] for node in merged_graph.nodes]], dtype=torch.float).t())
            elif isinstance(merged_graph.nodes[0][attr], (tuple, list)):
                node_attrs.append(torch.tensor([merged_graph.nodes[node][attr] for node in merged_graph.nodes], dtype=torch.float))
    data.x = torch.cat(node_attrs, dim=-1)
    
    # Set the edge index and edge features
    edge_index = torch.tensor(list(merged_graph.edges), dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor([list(merged_graph.edges[edge].values()) for edge in merged_graph.edges], dtype=torch.float)
    data.edge_index = edge_index
    data.edge_attr = edge_attr
    
    # Get the last 7 columns of the filtered row
    output_data = row.iloc[:, -7:]
    
    # Add the output data to the PyG graph data
    for col in output_data.columns:
        data[col] = output_data[col].values[0]
    
    return data

In [14]:
def to_pyg_graph_noxyz(name, gen_xyz_path, gen_csv_path):
    # Read the csv file
    try:
        df = pd.read_csv(f"{gen_csv_path}.csv")
    except FileNotFoundError:
        print(f"No csv file found at {gen_csv_path}.csv")
        return None
    
    # Filter the csv data to get the row associated with the name
    row = df[df['name'] == name]
    
    # Check if the row is empty
    if row.empty:
        print(f"No data found for name {name} in the csv file {gen_csv_path}.csv")
        return None
    
    # Extract the first two letters of the name which symbolize the element
    element = re.match(r"^[A-Za-z]{2}", name).group()
    
    # Construct the rest of the name
    rest_of_the_name = re.sub(r"^[A-Za-z]{2}", "", name)
    
    # Construct the filename for the xyz data
    filename = f"{element}_II{rest_of_the_name}_LS.xyz"
    
    filepath = f"{gen_xyz_path}/{filename}"
    if os.path.exists(filepath):
        # Create the MolGraph object for the file
        mg = MolGraph()
        mg.read_xyz(filepath)

        # Convert the molecular graph to the NetworkX graph
        G = to_networkx_graph(mg)

        # Get element properties for each node and store them in the node features
        for node in G.nodes:
            element_symbol = G.nodes[node]['element']
            element_properties = get_element_properties(element_symbol)
            G.nodes[node]['element_properties'] = element_properties
    else:
        print(f"No xyz file found at {filepath}")
        return None
    
    # Create the PyTorch Geometric Data object
    data = Data()
    
    # Set the node features directly from the element properties
    data.x = torch.tensor([G.nodes[node]['element_properties'] for node in G.nodes], dtype=torch.float)
    
    # Set the edge index without edge features
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    data.edge_index = edge_index
    
    # Get the last 7 columns of the filtered row
    output_data = row.iloc[:, -7:]
    
    # Add the output data to the PyG graph data
    for col in output_data.columns:
        data[col] = output_data[col].values[0]
    
    return data

In [15]:
def clean_xyz_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.xyz'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Remove stoichiometry and whatever comes after in the second line
            if len(lines) > 1:
                second_line_parts = lines[1].split('|')
                cleaned_second_line = second_line_parts[0] + '\n'
                lines[1] = cleaned_second_line

            # Write the cleaned lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(lines)

# # Replace 'your_folder_path' with the actual path to your folder
# clean_xyz_files('Data/Cambridge/X1/')

In [65]:
def to_pyg_graph(name, filepath):

    HS_path = "./data/Fe_Boryls_HS.csv"
    LS_path = "./data/Fe_Boryls_LS.csv"
    pattern = r'_(\d+)\.'

    # Search for the pattern in the file name
    match = re.search(pattern, name)

    # Extract the number if found
    number = int(match.group(1))
    
    try:
        HS_df = pd.read_csv(HS_path)
    except FileNotFoundError:
        print(f"No csv file found at {HS_path}")
        return None
    HS_row = HS_df[HS_df['XXXX'] == number]

    # Check if the row is empty
    if HS_row.empty:
        print(f"No data found for name {name} in the csv file {HS_path}")
        return None
    
    try:
        LS_df = pd.read_csv(LS_path)
    except FileNotFoundError:
        print(f"No csv file found at {LS_path}")
        return None
    LS_row = LS_df[LS_df['XXXX'] == number]

    # Check if the row is empty
    if LS_row.empty:
        print(f"No data found for name {name} in the csv file {LS_path}")
        return None


    # Check if the file exists
    if os.path.exists(filepath):
        # Create the MolGraph object for the file
        mg = MolGraph()
        mg.read_xyz(filepath)

        # Convert the molecular graph to the NetworkX graph
        G = to_networkx_graph(mg)

        # Get element properties for each node and store them in the node features
        for node in G.nodes:
            element_symbol = G.nodes[node]['element']
            element_properties = get_element_properties(element_symbol)
            G.nodes[node]['element_properties'] = element_properties
    else:
        print(f"No xyz file found at {filepath}")
        return None
    
    # Create the PyTorch Geometric Data object
    data = Data()
    
    # Set the node features directly from the element properties
    data.x = torch.tensor([G.nodes[node]['element_properties'] for node in G.nodes], dtype=torch.float)
    
    # Set the edge index without edge features
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    data.edge_index = edge_index
    HS_data = HS_row.iloc[:, 1:]
    LS_data = LS_row.iloc[:, 1:]
    for col in HS_data.columns:
        data[f"HS_{col}"] = HS_data[col].values[0]
    for col in LS_data.columns:
        data[f"LS_{col}"] = LS_data[col].values[0]
    
    return data

In [66]:
abemax = to_pyg_graph("monosubstituted_0001.xyz",'data/xyz_molsimp/monosubstituted_0001.xyz')

In [67]:
print(abemax.x.shape)
print(abemax.x[1,:])
print(abemax.edge_index.shape)
print(abemax)

torch.Size([35, 4])
tensor([0.0424, 0.0368, 2.0400, 0.8400])
torch.Size([2, 34])
Data(x=[35, 4], edge_index=[2, 34], HS_SCF_3=-3162.94937279, HS_SCF_2=-3163.11569879, HS_Delta_SCF=-436688.9130000693, HS_E_red=0.2359772296219027, LS_SCF_3=-3162.95788289, LS_SCF_2=-3163.09256706, LS_Delta_SCF=-353613.2883353355, LS_E_red=-0.6250439100861744)


In [75]:
# Assuming `data` is the Data object created previously
def save_graph_data(data, file_path):
    torch.save(data, file_path)
    print(f"Graph data saved to {file_path}")

In [80]:
# Directory containing the XYZ files
directory_path = '/Users/victorsu-ortiz/Desktop/Fe-Boryl_complexes/data/xyz_molsimp'

# List all files in the directory
file_names = [f for f in os.listdir(directory_path) if f.endswith('.xyz')]

# Read each file and store the data
data = []

for idx, file_name in enumerate(file_names):
    file_path = os.path.join(directory_path, file_name)
    tensor = to_pyg_graph(file_name, file_path)
    # # Define the regex pattern to extract the number before the file extension
    # pattern = r'_(\d+)\.'

    # # Search for the pattern in the file name
    # match = re.search(pattern, file_name)

    # # Extract the number if found
    # number = match.group(1)
    # edge_indices.append({"id": number, "edge_index": edge_index})
    save_path = f'./data/torch_processed/{file_name.split(".")[0]}.pth'
    

    # tensor = torch.tensor(coordinates)
    if tensor is not None:
        data.append((file_name, tensor))
        save_graph_data(tensor, save_path)

# Example of accessing the data
for file_name, tensor in data:
    print(f'File: {file_name}')
    print(f'Coordinates tensor: {tensor}')
    print()

Graph data saved to ./data/torch_processed/monosubstituted_0528.pth
Graph data saved to ./data/torch_processed/monosubstituted_0272.pth
Graph data saved to ./data/torch_processed/monosubstituted_0514.pth
No data found for name monosubstituted_0500.xyz in the csv file ./data/Fe_Boryls_HS.csv
No data found for name monosubstituted_0266.xyz in the csv file ./data/Fe_Boryls_HS.csv
Graph data saved to ./data/torch_processed/monosubstituted_0299.pth
Graph data saved to ./data/torch_processed/monosubstituted_0058.pth
Graph data saved to ./data/torch_processed/monosubstituted_0716.pth
Graph data saved to ./data/torch_processed/monosubstituted_0070.pth
Graph data saved to ./data/torch_processed/monosubstituted_0064.pth
No data found for name monosubstituted_0702.xyz in the csv file ./data/Fe_Boryls_HS.csv
No data found for name monosubstituted_0931.xyz in the csv file ./data/Fe_Boryls_HS.csv
No data found for name monosubstituted_0925.xyz in the csv file ./data/Fe_Boryls_HS.csv
No data found fo

In [82]:
len(data)
for d in data:
    print(d)

('monosubstituted_0528.xyz', Data(x=[32, 4], edge_index=[2, 32], HS_SCF_3=-3620.30233617, HS_SCF_2=-3620.49609436, HS_Delta_SCF=-508712.1278442085, HS_E_red=0.982447819290132, LS_SCF_3=-3620.35583283, LS_SCF_2=-3620.52324857, LS_Delta_SCF=-439550.0253694661, LS_E_red=0.265630671808738))
('monosubstituted_0272.xyz', Data(x=[28, 4], edge_index=[2, 29], HS_SCF_3=-3329.9123659, HS_SCF_2=-3330.09930169, HS_Delta_SCF=-490799.9166450179, HS_E_red=0.7968001932426583, LS_SCF_3=-3329.92575086, LS_SCF_2=-3330.11477844, LS_Delta_SCF=-496291.9112903389, LS_E_red=0.8537209026308632))
('monosubstituted_0514.xyz', Data(x=[30, 4], edge_index=[2, 30], HS_SCF_3=-3753.51639238, HS_SCF_2=-3753.70876181, HS_Delta_SCF=-505065.93846466264, HS_E_red=0.9446575992606376, LS_SCF_3=-3753.54050116, LS_SCF_2=-3753.72751218, LS_Delta_SCF=-490997.4330094028, LS_E_red=0.7988473131512961))
('monosubstituted_0299.xyz', Data(x=[27, 4], edge_index=[2, 26], HS_SCF_3=-3086.87707001, HS_SCF_2=-3087.06529865, HS_Delta_SCF=-494